<a href="https://colab.research.google.com/github/hdemma/hdemma.github.io/blob/master/Macro_Prediction_Models/ML_for_MapMatching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import pickle
from  more_itertools import unique_everseen
import tqdm

In [22]:
Vehicle_Type = 'Electric_Vehicles'
# Path = f'../DataSet/{Vehicle_Type}/'
# Path = f'Post_Covid_Data/'
Path = f''
Vehicle_ID = [751]
Train = False

In [23]:
def Data_Prep(df2):

    df = df2.drop('Unix_Timestamp_ms', axis=1)
    df = df.drop('F_Class', axis=1)
    df = df.drop('Time_US_Central', axis=1)
    df = df.drop("Street_Name", axis=1)
    df = df.drop("SOC", axis=1)

    df = df.dropna()
    DataSet = np.array(df, dtype=float)

    Tupple = np.shape(DataSet)  

    Number_of_Records = Tupple[0]
    Number_of_Features = Tupple[1]

    print(f'\nNumber_of_Records = {Number_of_Records}')
    print(f'Number_of_Features = {Number_of_Features}')


    Sample_Inputs = np.zeros((Number_of_Records, Number_of_Features - 1))

    for t in range(Number_of_Records):
        Sample_Inputs[t] = DataSet[t, :(Number_of_Features - 1)]


    Sample_Label = np.zeros((Number_of_Records,))

    for t in range(Number_of_Records):
        Sample_Label[t] = DataSet[t][Number_of_Features - 1]

    print(f'Train Label_Size = {np.shape(Sample_Label)}')

    return Sample_Inputs, Sample_Label, Number_of_Features

In [24]:
def DecisionTree_Model(Train):
    if Train == True:

        DT_regressor = DecisionTreeRegressor(max_depth=Number_of_Features,
                                             splitter='best',
                                             criterion="mse", min_samples_leaf=4)
        DT_regressor.fit(X_train, y_train)
        y_pred = DT_regressor.predict(X_train)

        MSE = np.sum(((y_train - y_pred) ** 2) / len(y_train))
        print(f'MSE on Train = {MSE}')

        y_pred = DT_regressor.predict(X_test)

        MSE = np.sum(((y_test - y_pred) ** 2) / len(y_test))
        print(f'MSE on Test = {MSE}')

        RMSE = np.sqrt(np.sum(((y_test - y_pred) ** 2) / len(y_test)))
        MAE = np.sum(abs(y_test - y_pred) / len(y_test))


        score = DT_regressor.score(X_test, y_test)
        print(f'Score = {score}\n\n')
        score = score * 100

        # save the model to disk
        filename = '0.00065_finalized_DT_model.sav'
        pickle.dump(DT_regressor, open(filename, 'wb'))
    else:
        #load the model from disk
        filename = '0.00065_finalized_DT_model.sav'
        loaded_model = pickle.load(open(filename, 'rb'))
        y_pred = loaded_model.predict(X)
        score = loaded_model.score(X,y)
        print(f'Score = {score}')

    MSE = np.sum(((y - y_pred) ** 2) / len(y))
    print(f'MSE on Test = {MSE}')

    return y_pred

In [25]:
def CheckErrorRate():
    Predicted_Final_Outcome = []
    False_Prediction = []
    index = 0
    while index < len(Location_Index):
        tmp_predicted = []
        final_outcome = []
        tmp_actual = []
        tmp_false = []
        for loc_count in range(index,len(Location_Index)):

            if loc_count == len(Location_Index)-1:
                tmp_predicted.append(Predicted_Values[loc_count])
                tmp_actual.append(Actual_Values[loc_count])
                final_outcome.append(0)
                tmp_false.append(0)
                index = loc_count

                m = tmp_predicted.index(max(tmp_predicted))
                final_outcome[m] = 1
                if final_outcome[m]!=tmp_actual[m]:
                    tmp_false[m]=1
                for f in final_outcome:
                    Predicted_Final_Outcome.append(f)
                for f in tmp_false:
                    False_Prediction.append(f)
                break

            elif Location_Index[index]==Location_Index[loc_count]:
                tmp_predicted.append(Predicted_Values[loc_count])
                tmp_actual.append(Actual_Values[loc_count])
                final_outcome.append(0)
                tmp_false.append(0)


            elif Location_Index[index] != Location_Index[loc_count]:
                m = tmp_predicted.index(max(tmp_predicted))
                final_outcome[m]=1
                if final_outcome[m] != tmp_actual[m]:
                    tmp_false[m] = 1
                for f in final_outcome:
                    Predicted_Final_Outcome.append(f)
                for f in tmp_false:
                    False_Prediction.append(f)
                index = loc_count-1

                break

        if index>len(Location_Index):
            break
        index=index+1


    ErrorPercentage = round(((sum(False_Prediction) / len(set(Location_Index))) * 100), 5)
    Error_Rate.append(ErrorPercentage)
    accuracy = round((100 - ErrorPercentage), 5)
    Accuracy.append(accuracy)
    
    print(sum(False_Prediction), len(set(Location_Index)), ErrorPercentage, accuracy)
    print(Error_Rate)
    print(Accuracy)
    return Predicted_Final_Outcome

In [26]:
def Final_Merge(df_pred, df2):
    Location_Index_test = list(df_pred.Location_Index)
    Predicted_Final_Test = list(df_pred.Predicted_Final)

    Location_Index_test = list(unique_everseen(Location_Index_test))

    csv = f"{Path}{Vehicle_Name}_Data_for_Map_Matching_AfterSplit.csv"
    df2 = pd.read_csv(csv, low_memory=False)
    print(f'len of df2 = {len(df2)}')

    Col = ['Distance_from_location', 'Average_Forward_Location',
           'Average_Backward_Location', 'Max_Forward_Location',
           'Max_Backward_Location', 'Min_Forward_Location',
           'Min_Backward_Location']
    for col in Col:
        df2 = df2.drop(col, axis=1)

    # print(df2.columns)

    Location_Index = df2.Location_Index

    # print(len(set(Location_Index)))

    Final_Output = df2.Final_Output

    Predicted_Final = []

    Location_Index_test = set(Location_Index_test)

    for i in tqdm.tqdm(range(len(Location_Index))):
        if Location_Index[i] not in Location_Index_test:
            Predicted_Final.append(Final_Output[i])

    A = Predicted_Final + Predicted_Final_Test

    df2['Predicted_Final'] = A
    Predicted_Final = df2.Predicted_Final

    print(df2.columns)

    df2 = df2.drop(df2[df2.Predicted_Final < 1].index)

    print(len(df2))
    csv = f"{Path}{Vehicle_Name}_Predicted_Data_for_Map_Matching.csv"
    df2.to_csv(csv, index=False)
    

In [27]:
for v in Vehicle_ID:
    Vehicle_Name = f'BYD_{v}'
    print(f'Processing {Vehicle_Name}')


    csv = f"{Vehicle_Name}_Data_for_Map_Matching_AfterSplit.csv"
    df2=pd.read_csv(csv,low_memory=False)
    print(len(df2))

    Sample_Inputs, Sample_Label, Number_of_Features = Data_Prep(df2)

    X = Sample_Inputs
    y = Sample_Label


    Candidates = []
    OSM_ID = []
    Lat_Long = []
    Location_Index = []

    for i in X:
        Point = (i[1],i[2])
        Lat_Long.append(Point)
        Candidates.append(i[3])
        OSM_ID.append(i[4])
        Location_Index.append(i[0])

    X = np.delete(X, 0, 1) #
    X = np.delete(X, 2, 1)


    Error_Rate = []
    Accuracy = []
    Actual_Values = y
    Train = False
    Predicted_Values = DecisionTree_Model(Train)
    Predicted_Final_Outcome= CheckErrorRate()

    Predicted_Fuel_Consumption = pd.DataFrame({"Location_Index": Location_Index,
                                               "Location": Lat_Long,
                                               "Candidate": Candidates,
                                               "OSM_ID": OSM_ID,
                                               "Predicted_Values": Predicted_Values,
                                               "Predicted_Final": Predicted_Final_Outcome,
                                               "Actual_Values":Actual_Values})

    csv = f"Predicted_{Vehicle_Name}_Data_for_Map_Matching.csv"

    Predicted_Fuel_Consumption.to_csv(f'{csv}', index=False)
    print('\n\n***********Mapping Done************\n')

    Final_Merge(Predicted_Fuel_Consumption,df2)

    print(f'Final Merge Done for {Vehicle_Name}')



Processing BYD_751
151968

Number_of_Records = 151947
Number_of_Features = 29
Train Label_Size = (151947,)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Score = 0.874765132423057
MSE on Test = 0.03114070138397227
1494 81539 1.83225 98.16775
[1.83225]
[98.16775]


***********Mapping Done************



  0%|          | 0/151968 [00:00<?, ?it/s]

len of df2 = 151968


100%|██████████| 151968/151968 [00:02<00:00, 62944.28it/s]


Index(['Location_Index', 'Latitude', 'Longitude', 'Unix_Timestamp_ms',
       'Time_US_Central', 'SOC', 'Candidate', 'OSM_ID', 'Street_Name',
       'F_Class', 'Bridge', 'Primary', 'Primary_link', 'Secondary',
       'Secondary_link', 'Tertiary', 'Tertiary_link', 'Trunk', 'Motorway',
       'Motorway_link', 'Service', 'Residential', 'Track', 'Unknown',
       'Unclassified', 'Final_Count', 'Final_Output', 'Predicted_Final'],
      dtype='object')
81553
Final Merge Done for BYD_751
